![](https://images.unsplash.com/photo-1602084551218-a28205125639?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2070&q=80)

<div class = 'alert alert-block alert-info'
     style = 'background-color:#4c1c84;
              color:#eeebf1;
              border-width:5px;
              border-color:#4c1c84;
              font-family:Comic Sans MS;
              border-radius: 50px 50px'>
    <p style = 'font-size:24px'>Exp 050</p>
    <a href = "#Config"
       style = "color:#eeebf1;
                font-size:14px">1.Config</a><br>
    <a href = "#Settings"
       style = "color:#eeebf1;
                font-size:14px">2.Settings</a><br>
    <a href = "#Data-Load"
       style = "color:#eeebf1;
                font-size:14px">3.Data Load</a><br>
    <a href = "#Pytorch-Settings"
       style = "color:#eeebf1;
                font-size:14px">4.Pytorch Settings</a><br>
    <a href = "#Training"
       style = "color:#eeebf1;
                font-size:14px">5.Training</a><br>
</div>

<p style = 'font-size:24px;
            color:#4c1c84'>
    実施したこと
</p>
    <li style = "color:#4c1c84;
                font-size:14px">GPT2-LayerConcat</li>

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Config
</h1>
<br>

In [1]:
import sys
sys.path.append("../src/utils/iterative-stratification/")
sys.path.append("../src/utils/detoxify")
sys.path.append("../src/utils/coral-pytorch/")
sys.path.append("../src/utils/pyspellchecker")

In [2]:
!pip install --no-index --find-links ../src/utils/faiss/ faiss-gpu==1.6.3

Looking in links: ../src/utils/faiss/


In [3]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
gc.enable()
import sys
import glob
import copy
import math
import time
import random
import string
import psutil
import pathlib
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict
from box import Box
from typing import Optional
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

from tqdm.auto import tqdm as tqdmp
from tqdm.autonotebook import tqdm as tqdm
tqdmp.pandas()

## Model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, AutoModelForSequenceClassification
from transformers import RobertaModel, RobertaForSequenceClassification
from transformers import RobertaTokenizer
from transformers import LukeTokenizer, LukeModel, LukeConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, BertForMaskedLM
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import DebertaTokenizer, DebertaModel
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config

# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning import LightningDataModule, LightningDataModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers.csv_logs import CSVLogger
from pytorch_lightning.callbacks import RichProgressBar

from sklearn.linear_model import Ridge
from sklearn.svm import SVC, SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import rankdata
from cuml.svm import SVR as cuml_SVR
from cuml.linear_model import Ridge as cuml_Ridge
import cudf
from detoxify import Detoxify
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from ast import literal_eval
from nltk.tokenize import TweetTokenizer
import spacy
from scipy.stats import sem
from copy import deepcopy
from spellchecker import SpellChecker
from typing import Text, Set, List

import faiss
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer as cuTfidfVectorizer
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
from cuml.decomposition.tsvd import TruncatedSVD as cuTruncatedSVD

2022-02-05 04:44:57.132602: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
import torch

config = {
    "exp_comment":"Psuedo Labeling",
    "seed": 42,
    "root": "/content/drive/MyDrive/kaggle/Jigsaw/raw",
    "n_fold": 4,
    "epoch": 5,
    "max_length": 256,
    "environment": "AWS",
    "project": "Jigsaw",
    "entity": "dataskywalker",
    "exp_name": "050_exp",
    "margin": 0.5,
    "train_fold": [0, 1, 2, 3],

    "trainer": {
        "gpus": 1,
        "accumulate_grad_batches": 8,
        "progress_bar_refresh_rate": 100,
        "fast_dev_run": True,
        "num_sanity_val_steps": 0,
    },

    "train_loader": {
        "batch_size": 4,
        "shuffle": True,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": True,
    },

    "valid_loader": {
        "batch_size": 2,
        "shuffle": False,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": False,
    },

    "test_loader": {
        "batch_size": 1,
        "shuffle": False,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": False,
    },

    "backbone": {
        "name": "gpt2-medium",
        "output_dim": 1,
    },

    "optimizer": {
        "name": "torch.optim.AdamW",
        "params": {
            "lr": 1e-6,
        },
    },

    "scheduler": {
        "name": "torch.optim.lr_scheduler.CosineAnnealingWarmRestarts",
        "params": {
            "T_0": 20,
            "eta_min": 0,
        },
    },

    "loss": "nn.MarginRankingLoss",
}

config = Box(config)
config.tokenizer = GPT2Tokenizer.from_pretrained(
    config.backbone.name, 
    bos_token='<|startoftext|>', 
    eos_token='<|endoftext|>', 
    pad_token='<|pad|>',
    add_prefix_space = True,
    lowercase=True,
)
config.tokenizer.pad_token = config.tokenizer.eos_token
config.tokenizer.padding_side = "right"

config.model = GPT2Model.from_pretrained(config.backbone.name)

# pprint(config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# config.tokenizer.save_pretrained(f"../data/processed/{config.backbone.name}")
# pretrain_model = RobertaModel.from_pretrained(config.backbone.name)
# pretrain_model.save_pretrained(f"../data/processed/{config.backbone.name}")

config.tokenizer.save_pretrained(f"../data/processed/{config.backbone.name}")
config.model.save_pretrained(f"../data/processed/{config.backbone.name}")

In [6]:
# 個人的にAWSやKaggle環境やGoogle Colabを行ったり来たりしているのでまとめています
import os
import sys
from pathlib import Path

if config.environment == 'AWS':
    
    INPUT_DIR = Path('/mnt/work/data/kaggle/Jigsaw/')
    MODEL_DIR = Path(f'../models/{config.exp_name}/')
    OUTPUT_DIR = Path(f'../data/interim/{config.exp_name}/')
    UTIL_DIR = Path('/mnt/work/shimizu/kaggle/PetFinder/src/utils')
    
    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"Your environment is 'AWS'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}\nUTIL_DIR is {UTIL_DIR}")
    
    
elif config.environment == 'Kaggle':
    INPUT_DIR = Path('../input/*****')
    MODEL_DIR = Path('./')
    OUTPUT_DIR = Path('./')
    print(f"Your environment is 'Kaggle'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}")

    
elif config.environment == 'Colab':
    INPUT_DIR = Path('/content/drive/MyDrive/kaggle/Jigsaw/raw')
    BASE_DIR = Path("/content/drive/MyDrive/kaggle/Jigsaw/interim")

    MODEL_DIR = BASE_DIR / f'{config.exp_name}'
    OUTPUT_DIR = BASE_DIR / f'{config.exp_name}/'

    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    if not os.path.exists(INPUT_DIR):
        print('Please Mount your Google Drive.')
    else:
        print(f"Your environment is 'Colab'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}")
        
else:
    print("Please choose 'AWS' or 'Kaggle' or 'Colab'.\nINPUT_DIR is not found.")

Your environment is 'AWS'.
INPUT_DIR is /mnt/work/data/kaggle/Jigsaw
MODEL_DIR is ../models/050_exp
OUTPUT_DIR is ../data/interim/050_exp
UTIL_DIR is /mnt/work/shimizu/kaggle/PetFinder/src/utils


In [7]:
# Seed固定
seed_everything(config.seed)

42

In [8]:
## 処理時間計測
@contextmanager
def timer(name:str, slack:bool=False):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    print(f'<< {name} >> Start')
    yield
    
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    
    print(f"<< {name} >> {m1:.1f}GB({sign}{delta:.1f}GB):{time.time() - t0:.1f}sec", file=sys.stderr)

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Data Load
</h1>
<br>

In [9]:
## Data Check
for dirnames, _, filenames in os.walk(INPUT_DIR):
    
    for filename in filenames:

        print(f'{dirnames}/{filename}')

/mnt/work/data/kaggle/Jigsaw/comments_to_score.csv
/mnt/work/data/kaggle/Jigsaw/sample_submission.csv
/mnt/work/data/kaggle/Jigsaw/validation_data.csv


In [10]:
val_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/validation_data.csv")
test_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/comments_to_score.csv")

display(val_df.head())
display(test_df.head())

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [11]:
with timer("Count less text & more text"):
    less_df = val_df.groupby(["less_toxic"])["worker"].agg("count").reset_index()
    less_df.columns = ["text", "less_count"]

    more_df = val_df.groupby(["more_toxic"])["worker"].agg("count").reset_index()
    more_df.columns = ["text", "more_count"]
    
    text_df = pd.merge(
        less_df,
        more_df,
        on="text",
        how="outer"
    )

    text_df["less_count"] = text_df["less_count"].fillna(0)
    text_df["more_count"] = text_df["more_count"].fillna(0)
    
    text_df["target"] = text_df["more_count"]/(text_df["less_count"] + text_df["more_count"])
    
    display(text_df)

<< Count less text & more text >> Start


,text,less_count,more_count,target
0,\n\nThe comment directly above this one are fr...,1.0,2.0,0.666667
1,\n\nwhy should people have to read crap posted...,1.0,8.0,0.888889
2,\nGo F yourself you cottonheadednittymuggins.,1.0,2.0,0.666667
3,"\nU POUR ADMIN, U UPDATE VANDAL COUNT, WHILE R...",3.0,0.0,0.000000
4,\n\n koreans claim \n\ni see youve seen the ...,4.0,2.0,0.333333
...,...,...,...,...
14246,{{unblock|The reason I used sockpuppets was to...,0.0,3.0,1.000000
14247,{{unblock}} BECAUSE BAIL OUT IS THE BEST FUCK...,0.0,3.0,1.000000
14248,|Christopher Connor]]. I am off to my daily ma...,0.0,6.0,1.000000
14249,}} \n\nSomeone should address this fucking pro...,0.0,1.0,1.000000


<< Count less text & more text >> 3.2GB(+0.0GB):0.1sec


<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Make Fold
</h1>
<br>

In [12]:
# texts = set(val_df["less_toxic"].to_list() + val_df["more_toxic"].to_list())
# text2id = {t:id for id,t in enumerate(texts)}
# val_df['less_id'] = val_df['less_toxic'].map(text2id)
# val_df['more_id'] = val_df['more_toxic'].map(text2id)
# val_df.head()

In [13]:
# len_ids = len(text2id)
# idarr = np.zeros((len_ids, len_ids), dtype=bool)

# for lid, mid in val_df[['less_id', 'more_id']].values:
#     min_id = min(lid, mid)
#     max_id = max(lid, mid)
#     idarr[max_id, min_id] = True

In [14]:
# def add_ids(i, this_list):
#     for j in range(len_ids):
#         if idarr[i, j]:
#             idarr[i, j] = False
#             this_list.append(j)
#             this_list = add_ids(j,this_list)
#             #print(j,i)
#     for j in range(i+1,len_ids):
#         if idarr[j, i]:
#             idarr[j, i] = False
#             this_list.append(j)
#             this_list = add_ids(j,this_list)
#             #print(j,i)
#     return this_list

# group_list = []
# for i in tqdm(range(len_ids)):
#     for j in range(i+1,len_ids):
#         if idarr[j, i]:
#             this_list = add_ids(i,[i])
# #             print(this_list)
#             group_list.append(this_list)

# id2groupid = {}
# for gid,ids in enumerate(group_list):
#     for id in ids:
#         id2groupid[id] = gid

# val_df['less_gid'] = val_df['less_id'].map(id2groupid)
# val_df['more_gid'] = val_df['more_id'].map(id2groupid)

In [15]:
# print('unique text counts:', len_ids)
# print('grouped text counts:', len(group_list))

In [16]:
# # now we can use GroupKFold with group id
# group_kfold = GroupKFold(n_splits=config.n_fold)

# # Since df.less_gid and df.more_gid are the same, let's use df.less_gid here.
# for fold, (trn, val) in enumerate(group_kfold.split(val_df, val_df, val_df.less_gid)): 
#     val_df.loc[val , "fold"] = fold

# val_df["fold"] = val_df["fold"].astype(int)
# val_df

In [17]:
# val_df.to_csv(OUTPUT_DIR/"val_df.csv", index=False)

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Pytorch Dataset
</h1>
<br>

In [18]:
val_df = pd.read_csv(OUTPUT_DIR/"val_df.csv")
display(val_df.head())

,worker,less_toxic,more_toxic,less_id,more_id,less_gid,more_gid,fold
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,14087,10662,1878,1878,1
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,13499,3448,485,485,2
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",12117,8697,3754,3754,1
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,7229,1266,1042,1042,2
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",4458,4960,1951,1951,1


In [19]:
class JigsawDataset:
    
    def __init__(self, df, tokenizer, max_length, mode):
        
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.mode = mode
        
        if self.mode == "train":
            self.more_toxic = df["more_toxic"].values
            self.less_toxic = df["less_toxic"].values
            
        elif self.mode == "valid":
            self.more_toxic = df["more_toxic"].values
            self.less_toxic = df["less_toxic"].values
            
        else:
            self.text = df["text"].values
            
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        
        if self.mode == "train":
            
            more_toxic = self.more_toxic[index]
            less_toxic = self.less_toxic[index]

            inputs_more_toxic = self.tokenizer(
                '<|startoftext|>'+ more_toxic + '<|endoftext|>',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=True,
                max_length = self.max_len,
                padding="max_length",
            )

            inputs_less_toxic = self.tokenizer(
                '<|startoftext|>'+ less_toxic + '<|endoftext|>',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=True,
                max_length = self.max_len,
                padding="max_length",
            )
            
            target = 1

            more_toxic_ids = inputs_more_toxic["input_ids"]
            more_toxic_mask = inputs_more_toxic["attention_mask"]

            less_toxic_ids = inputs_less_toxic["input_ids"]
            less_toxic_mask = inputs_less_toxic["attention_mask"]
            
            return {
                'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
                'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
                
                'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
                'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
                
                'target': torch.tensor(target, dtype=torch.float)
            }
        
        elif self.mode == "valid":
            
            more_toxic = self.more_toxic[index]
            less_toxic = self.less_toxic[index]

            inputs_more_toxic = self.tokenizer(
                '<|startoftext|>'+ more_toxic + '<|endoftext|>',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=True,
                max_length = self.max_len,
                padding="max_length",
            )

            inputs_less_toxic = self.tokenizer(
                '<|startoftext|>'+ less_toxic + '<|endoftext|>',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=True,
                max_length = self.max_len,
                padding="max_length",
            )
            
            target = 1

            more_toxic_ids = inputs_more_toxic["input_ids"]
            more_toxic_mask = inputs_more_toxic["attention_mask"]

            less_toxic_ids = inputs_less_toxic["input_ids"]
            less_toxic_mask = inputs_less_toxic["attention_mask"]
            
            return {
                'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
                'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
                
                'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
                'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
                
                'target': torch.tensor(target, dtype=torch.float)
            }
        
        else:
            
            text = self.text[index]

            inputs_text = self.tokenizer(
                '<|startoftext|>'+ text + '<|endoftext|>',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=True,
                max_length = self.max_len,
                padding="max_length",
            )
            
            text_ids = inputs_text["input_ids"]
            text_mask = inputs_text["attention_mask"]

            return {
                'text_ids': torch.tensor(text_ids, dtype=torch.long),
                'text_mask': torch.tensor(text_mask, dtype=torch.long),
            }

<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal;
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center; 
             border-radius: 100px 100px;">
    DataModule
</h2>
<br>

In [20]:
class JigsawDataModule(LightningDataModule):

    def __init__(self, train_df, valid_df, test_df, cfg):

        super().__init__()
        self._train_df = train_df
        self._valid_df = valid_df
        self._test_df = test_df
        self._cfg = cfg

    def train_dataloader(self):
        dataset = JigsawDataset(
            df=self._train_df, 
            tokenizer=self._cfg.tokenizer,
            max_length=self._cfg.max_length,
            mode="train",
            )
        return DataLoader(dataset, **self._cfg.train_loader)

    def val_dataloader(self):
        dataset = JigsawDataset(
            df=self._valid_df, 
            tokenizer=self._cfg.tokenizer,
            max_length=self._cfg.max_length,
            mode="valid",
            )
        return DataLoader(dataset, **self._cfg.valid_loader)

    def test_dataloader(self):
        dataset = JigsawDataset(
            df=self._test_df,
            tokenizer = self._cfg.tokenizer,
            max_length=self._cfg.max_length,
            mode="test",
        )

        return DataLoader(dataset, **self._cfg.test_loader)

In [21]:
## DataCheck
seed_everything(config.seed)

sample_dataloader = JigsawDataModule(val_df, val_df, test_df, config).train_dataloader()
for data in sample_dataloader:
    break

In [22]:
print(data["more_toxic_ids"].size())
print(data["more_toxic_mask"].size())


# print(data["more_toxic_ids"][1])
# print(data["more_toxic_mask"][1])


print(data["target"].size())
print(data["target"])

base_model = GPT2Model.from_pretrained(
    config.backbone.name,
    attn_pdrop=0,
    embd_pdrop=0,
    resid_pdrop=0,
    summary_first_dropout=0,
    output_hidden_states=True
)
base_model.resize_token_embeddings(len(config.tokenizer))

output = base_model(
    data["more_toxic_ids"],
    attention_mask=data["more_toxic_mask"],
)
sequence_output = torch.cat(
    [output["hidden_states"][-1*i][:,0] for i in range(1, 4+1)], dim=1
)

feature = nn.LayerNorm(1024*4)(sequence_output)
out = nn.Linear(1024*4, 1)(feature)

print(output["hidden_states"][-1].size())
print(sequence_output.size())
print(feature.size())
print(out.size())
print(out)

torch.Size([4, 256])
torch.Size([4, 256])
torch.Size([4])
tensor([1., 1., 1., 1.])
torch.Size([4, 256, 1024])
torch.Size([4, 4096])
torch.Size([4, 4096])
torch.Size([4, 1])
tensor([[1.0786],
        [1.0786],
        [1.0786],
        [1.0786]], grad_fn=<AddmmBackward>)


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal;
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center; 
             border-radius: 100px 100px;">
    LigitningModule
</h2>
<br>

In [23]:
def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=config.margin)(outputs1, outputs2, targets)

In [24]:
class JigsawModel(pl.LightningModule):
    
    def __init__(self, cfg, fold_num):
        
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self.criterion = criterion
        self.save_hyperparameters(cfg)
        self.fold_num = fold_num
        
    def __build_model(self):
        
        self.base_model = GPT2Model.from_pretrained(
            config.backbone.name,
            attn_pdrop=0,
            embd_pdrop=0,
            resid_pdrop=0,
            summary_first_dropout=0,
            output_hidden_states=True
        )
        self.base_model.resize_token_embeddings(len(self.cfg.tokenizer))
        self.base_model.config.pad_token_id = self.base_model.config.eos_token_id
        
        self.norm = nn.LayerNorm(1024*4)
        self.drop = nn.Dropout(p=0.3)
        self.head = nn.Linear(1024*4, 1)
        
    def forward(self, ids, mask):
        
        output = self.base_model(
            input_ids=ids, 
            attention_mask=mask,
            output_hidden_states=True,
        )
        
        sequence_output = torch.cat(
            [torch.mean(output["hidden_states"][-1*i], dim=1) for i in range(1, 4+1)], dim=1
        )
        
        feature = self.norm(sequence_output)
#         feature = self.drop(feature)
        out = self.head(sequence_output)
    
        
        return {
            "logits":out, 
        }
    
    def training_step(self, batch, batch_idx):
        
        more_toxic_ids = batch['more_toxic_ids']
        more_toxic_mask = batch['more_toxic_mask']
        
        less_toxic_ids = batch['less_toxic_ids']
        less_toxic_mask = batch['less_toxic_mask']
        
        targets = batch['target']
        
        more_outputs = self.forward(
            more_toxic_ids, 
            more_toxic_mask,
        )
        
        less_outputs = self.forward(
            less_toxic_ids, 
            less_toxic_mask,
        )
        
        loss = self.criterion(more_outputs["logits"], less_outputs["logits"], targets)

        return {
            "loss":loss,
            "targets":targets,
        }
    
    def training_epoch_end(self, training_step_outputs):

        loss_list = []

        for out in training_step_outputs:

            loss_list.extend([out["loss"].cpu().detach().tolist()])

        meanloss = sum(loss_list)/len(loss_list)

        logs = {f"train_loss/fold{self.fold_num+1}": meanloss,}

        self.log_dict(
            logs,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True
        )
        
    def validation_step(self, batch, batch_idx):

        more_toxic_ids = batch['more_toxic_ids']
        more_toxic_mask = batch['more_toxic_mask']
        
        less_toxic_ids = batch['less_toxic_ids']
        less_toxic_mask = batch['less_toxic_mask']
        
        targets = batch['target']

        more_outputs = self.forward(
            more_toxic_ids, 
            more_toxic_mask,
        )
        
        less_outputs = self.forward(
            less_toxic_ids, 
            less_toxic_mask,
        )

        
        outputs = more_outputs["logits"] - less_outputs["logits"]
        logits = outputs.clone()[:, 0]

        logits[logits > 0] = 1
        loss = nn.BCEWithLogitsLoss()(logits, targets)

        return {
            "loss":loss,
            "pred":outputs,
            "targets":targets,
        }
    
    
    def validation_epoch_end(self, validation_step_outputs):

        loss_list = []
        pred_list = []
        target_list = []

        for out in validation_step_outputs:
            loss_list.extend([out["loss"].cpu().detach().tolist()])
            pred_list.append(out["pred"][:, 0].detach().cpu().numpy())
            target_list.append(out["targets"].detach().cpu().numpy())

        meanloss = sum(loss_list)/len(loss_list)
        pred_list = np.concatenate(pred_list)
        pred_count = sum(x>0 for x in pred_list)/len(pred_list)

        logs = {
            f"valid_loss/fold{self.fold_num+1}":meanloss,
            f"valid_acc/fold{self.fold_num+1}":pred_count,
        }

        self.log_dict(
            logs,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True
        )
        
    def configure_optimizers(self):

        optimizer = eval(self.cfg.optimizer.name)(
            self.parameters(), **self.cfg.optimizer.params
        )

        self.scheduler = eval(self.cfg.scheduler.name)(
            optimizer, **self.cfg.scheduler.params
        )
        
        scheduler = {"scheduler": self.scheduler, "interval": "step",}

        return [optimizer], [scheduler]

<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal;
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center; 
             border-radius: 100px 100px;">
    Training
</h2>
<br>

In [25]:
# skf = StratifiedKFold(n_splits=config.n_fold, shuffle=True, random_state=config.seed)

# for fold, (_, val_idx) in enumerate(skf.split(X=val_df, y=val_df["worker"])):
#     val_df.loc[val_idx, "kfold"] = int(fold)

# val_df["kfold"] = val_df["kfold"].astype(int)
val_df.head()

,worker,less_toxic,more_toxic,less_id,more_id,less_gid,more_gid,fold
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,14087,10662,1878,1878,1
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,13499,3448,485,485,2
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",12117,8697,3754,3754,1
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,7229,1266,1042,1042,2
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",4458,4960,1951,1951,1


In [26]:
# ## Debug
# config.trainer.fast_dev_run = True
# config.backbone.output_dim = 1

# for fold in config.train_fold:
    
#     print("★"*25, f" Fold{fold+1} ", "★"*25)

#     df_train = val_df[val_df.fold != fold].reset_index(drop=True)
#     df_valid = val_df[val_df.fold != fold].reset_index(drop=True)

#     datamodule = JigsawDataModule(df_train, df_valid, test_df, config)
#     sample_dataloader = JigsawDataModule(df_train, df_valid, test_df, config).train_dataloader()

#     config.scheduler.params.T_0 = config.epoch * len(sample_dataloader)
#     model = JigsawModel(config, fold)
#     lr_monitor = callbacks.LearningRateMonitor()

#     loss_checkpoint = callbacks.ModelCheckpoint(
#         filename=f"best_acc_fold{fold+1}",
#         monitor=f"valid_acc/fold{fold+1}",
#         save_top_k=1,
#         mode="max",
#         save_last=False,
#         dirpath=MODEL_DIR,
#         save_weights_only=True,
#     )

#     wandb_logger = WandbLogger(
#         project=config.project, 
#         entity=config.entity,
#         name = f"{config.exp_name}",
#         tags = ['GPT2', "MarginRankLoss"]
#     )

#     lr_monitor = LearningRateMonitor(logging_interval='step')

#     trainer = pl.Trainer(
#         max_epochs=config.epoch,
#         callbacks=[loss_checkpoint, lr_monitor, RichProgressBar()],
# #         deterministic=True,
#         logger=[wandb_logger],
#         **config.trainer
#     )
#     trainer.fit(model, datamodule=datamodule)

In [ ]:
# Training
config.trainer.fast_dev_run = False
config.backbone.output_dim = 1

for fold in config.train_fold:
    
    print("★"*25, f" Fold{fold+1} ", "★"*25)

    df_train = val_df[val_df.fold != fold].reset_index(drop=True)
    df_valid = val_df[val_df.fold == fold].reset_index(drop=True)

    datamodule = JigsawDataModule(df_train, df_valid, test_df, config)
    sample_dataloader = JigsawDataModule(df_train, df_valid, test_df, config).train_dataloader()

    config.scheduler.params.T_0 = config.epoch * len(sample_dataloader)
    model = JigsawModel(config, fold)
    lr_monitor = callbacks.LearningRateMonitor()

    loss_checkpoint = callbacks.ModelCheckpoint(
        filename=f"best_acc_fold{fold+1}",
        monitor=f"valid_acc/fold{fold+1}",
        save_top_k=1,
        mode="max",
        save_last=False,
        dirpath=MODEL_DIR,
        save_weights_only=True,
    )

    wandb_logger = WandbLogger(
        project=config.project, 
        entity=config.entity,
        name = f"{config.exp_name}",
        tags = ['GPT2', "MarginRankLoss"]
    )

    lr_monitor = LearningRateMonitor(logging_interval='step')

    trainer = pl.Trainer(
        max_epochs=config.epoch,
        callbacks=[loss_checkpoint, lr_monitor, RichProgressBar()],
#         deterministic=True,
        logger=[wandb_logger],
        **config.trainer
    )
    trainer.fit(model, datamodule=datamodule)

Epoch 3    ━━━━━━━╸━━━━━━━━━━━━━━ 3280/9409 0:23:32 • 0:43:58 2.32it/s loss: 0.278 v_num:    
                                                                       wqal valid_loss/fold2:
                                                                       0.503 valid_acc/fold2:
                                                                       0.676                 
                                                                       train_loss/fold2:     
                                                                       0.045                 

★★★★★★★★★★★★★★★★★★★★★★★★★  Fold3  ★★★★★★★★★★★★★★★★★★★★★★★★★


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal;
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center; 
             border-radius: 100px 100px;">
    Inference
</h2>
<br>

In [ ]:
text_df = pd.DataFrame()
text_df["text"] = list(set(val_df["less_toxic"].unique().tolist() + val_df["more_toxic"].unique().tolist()))

display(text_df.head())
display(text_df.shape)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Device == {device}")

# attention_array = np.zeros((len(text_df), config.max_length)) # attention格納
# mask_array = np.zeros((len(text_df), config.max_length)) # mask情報格納,後でattentionと掛け合わせる
# feature_array = np.zeros((len(text_df), 768))
PRED = np.zeros(len(text_df))

for fold in config.train_fold:

    pred_list = []
    print("★"*25, f" Fold{fold+1} ", "★"*25)

    test_dataloader = JigsawDataModule(val_df, val_df, text_df, config).test_dataloader()
    model = JigsawModel(config, fold)

    loss_checkpoint = callbacks.ModelCheckpoint(
        filename=f"best_acc_fold{fold+1}",
        monitor=f"valid_acc/fold{fold+1}",
        save_top_k=1,
        mode="max",
        save_last=False,
        dirpath="../input/toxicroberta/",
    )
    model = model.load_from_checkpoint(MODEL_DIR/f"best_acc_fold{fold+1}.ckpt", cfg=config, fold_num=fold)
    model.to(device)
    model.eval()
    
    attention_list = []
    feature_list = []
    mask_list = []
    pred_list = []
    
    for step, data in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):

        text_ids = data["text_ids"].to(device)
        text_mask = data["text_mask"].to(device)
        
        mask_list.append(text_mask.detach().cpu().numpy())
            
        outputs = model(
            text_ids, 
            text_mask,
        )
                
        pred_list.append(outputs["logits"][:, 0].detach().cpu().numpy())

    PRED += np.concatenate(pred_list)/len(config.train_fold)
    
    

In [ ]:
text_df["target"] = PRED
text_df.to_pickle(OUTPUT_DIR/f"{config.exp_name}__text_df.pkl")
# np.save(OUTPUT_DIR/'toxic-attention.npy', attention_array)
# np.save(OUTPUT_DIR/'toxic-mask.npy', mask_array)
# np.save(OUTPUT_DIR/'toxic-feature.npy', feature_array)

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(text_df["target"], color="#4c1c84")
plt.grid()
plt.show()

<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal;
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center; 
             border-radius: 100px 100px;">
    Attention Load
</h2>
<br>

In [46]:
text_df = pd.read_pickle(OUTPUT_DIR/"text_df.pkl")
attention_array = np.load(OUTPUT_DIR/'toxic-attention.npy')
mask_array = np.load(OUTPUT_DIR/'toxic-mask.npy')

In [47]:
from IPython.display import display, HTML

def highlight_r(word, attn):
    html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}">{}</span>'.format(html_color, word)

In [49]:
num = 12

ids = config.tokenizer(text_df.loc[num, "text"])["input_ids"]
tokens = config.tokenizer.convert_ids_to_tokens(ids)
attention = attention_array[num, :][np.nonzero(mask_array[num, :])]

html_outputs = []

for word, attn in zip(tokens, attention):
    html_outputs.append(highlight_r(word, attn))

print(f"Offensive Score is {PRED[num]}")
display(HTML(' '.join(html_outputs)))
display(text_df.loc[num, "text"])

Offensive Score is -0.7636717192362994


'Please excuse those lines of events- my friend was borrowing my account, and he decided it would be clever of him to vandalise certaint pages. It was foolish of me to lend him my account, and it will not happen again. '

In [50]:
text_df.sort_values("target", ascending=False).head(20)

,text,target
960,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PI...,1.825503
9527,FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT ...,1.638090
5726,"FUCKING COCKSUCKING ADMINS, EAT MY SHIT CUNTS!...",1.599204
81,"FUCK YOU, YOU ATHEIST CUNT! FUCK YOU, YOU ATHE...",1.569043
12595,F*ck You \nShut the fuck up you whiney ass li...,1.496165
8763,Fuck u bitch \n\nFuck u Bitch,1.483061
13913,Fuck you asshole! \n\nGo fuck yourself! Dirty...,1.473651
5779,HEEY FUCKFACE YOU ARE A REAL BITCH ASS PUSSY W...,1.472036
3011,IN THE ASS BITCH FUCKING COCK SUCKER!,1.462103
775,FUCK YOU ALL!! FUCK YOU ALL!! FUCK YOU ALL!!...,1.460091


In [52]:
high_score_list = text_df.sort_values("target", ascending=False).head(20).index.tolist()

for num in high_score_list:
    
    ids = config.tokenizer(text_df.loc[num, "text"])["input_ids"]
    tokens = config.tokenizer.convert_ids_to_tokens(ids)
    attention = attention_array[num, :][np.nonzero(mask_array[num, :])]

    html_outputs = []

    for word, attn in zip(tokens, attention):
        html_outputs.append(highlight_r(word, attn))

    print(f"Offensive Score is {PRED[num]}")
    display(HTML(' '.join(html_outputs)))
    display(text_df.loc[num, "text"])



Offensive Score is 1.8255026936531067


' FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIECE OF SHIT '

Token indices sequence length is longer than the specified maximum sequence length for this model (1742 > 512). Running this sequence through the model will result in indexing errors


Offensive Score is 1.6380899846553802


'FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U USELESS BOT FUCK YOU U

...

' FUCK YOUR MOTHER \n\nFUCK OFF'

Offensive Score is 1.4202548265457153


' FUCK BILLCJ AND KILL BILLCJ BECAUSE I HATE HIS BIG FAT FUCK \n\nKILL BILLCJ!! KILL BILLCJ! KILL BILLCJS ASS! KILL HIS BIG FAT FUCK!! KILL HIS ASS! KILL BILLCJ! FUCK BILLCJ! KILL HIS ASS! KILL BILLCJ!!! KILL BILLCJ!! I HATE HIM! KILL BILLCJ AND AVENGE TOUGHHEAD!!! KILL HIS ASS!!!'

Offensive Score is 1.4180269241333008


'YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! YOU ARE A MOTHJER FUCKER COCKSUCKER! 